In [2]:
from tqdm import tqdm
from functools import reduce
import itertools
import collections
from collections.abc import Iterable
import time

In [27]:
# Basic tqdm usage
for i in tqdm(range(100)):
    time.sleep(0.01)

100%|██████████| 100/100 [00:01<00:00, 97.06it/s]


In [36]:
# It's possible to use tqdm with generators, but must add total number of iterations.
k = 0
for i in tqdm(map(lambda x: x*x, range(100)), total=100):
    k += i
    time.sleep(0.01)
k

100%|██████████| 100/100 [00:01<00:00, 96.72it/s]


328350

In [37]:
# Wrap loops in to a generator and use tqdm
k = 0
def gen():
    global k
    for i in range(10):
        for j in range(10):
            k += i*j
            time.sleep(0.01)
            yield

for _ in tqdm(gen(), total=100):
    pass
k

100%|██████████| 100/100 [00:01<00:00, 96.96it/s]


2025

In [3]:
# tqdm manually update

k = 0
pbar = tqdm(total=10)
for i in range(15):
    k += 1
    time.sleep(0.1)
    pbar.update(1)
    
pbar.close()
k

15it [00:01,  9.82it/s]                        


15

In [15]:
class MyIterable(object):
    def __init__(self, l):
        self.__l = l
    
    def __len__(self):
        return self.__l
    
    def __getitem__(self, idx):
        if idx >= len(self):
            raise IndexError
        return idx

class MyOtherIterable(object):
    """Iterable"""
    def __init__(self, l):
        self.__l = l
    
    def __iter__(self):
        for i in range(self.__l):
            yield i


In [16]:
[i for i in MyIterable(10)], [i for i in MyOtherIterable(10)]

([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], [0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [17]:
# Check whether something is iterable
# The only(?) reliable way to determine whether an object is iterable is to call iter(obj)
def is_iterable(x):
    try:
        iter(x)
        return True
    except:
        return False

def _is_iterable(x):
    """
    This does not check for classes that are iterable through __getitem__"""
    return isinstance(x, collections.abc.Iterable)

print("Is a list iterable?",  is_iterable([]),      _is_iterable([]))
print("Is a set iterable?",   is_iterable(set()),   _is_iterable(set()))
print("Is a tuple iterable?", is_iterable(tuple()), _is_iterable(tuple()))
print("Is a dict iterable?",  is_iterable(dict()),  _is_iterable(dict()))
print("Is a map object iterable?", is_iterable(map(lambda x: x, [1,2,3])), _is_iterable(map(lambda x: x, [1,2,3])))
print("Is MyIterable iterable?", is_iterable(MyIterable(10)), _is_iterable(MyIterable(10)))
print("Is MyOtherIterable iterable?", is_iterable(MyOtherIterable(10)), _is_iterable(MyOtherIterable(10)))
print("Is a number iterable?", is_iterable(1), _is_iterable(1))

Is a list iterable? True True
Is a set iterable? True True
Is a tuple iterable? True True
Is a dict iterable? True True
Is a map object iterable? True True
Is MyIterable iterable? True False
Is MyOtherIterable iterable? True True
Is a number iterable? False False


In [18]:
def select(l, indices):
    """Select from iterable using an index of a list of indices,
    returning a list with entries in the same order as their
    corresponding indices. Examples:

    """
    if is_iterable(indices):
        try:
            return [l[i] for i in indices]
        except TypeError:
            mask = indices_to_selection_mask(indices, max(indices))
            return compress(l, mask)
    else:
        i = indices
        try:
            return l[i]
        except TypeError:
            return next(itertools.islice(l, i, None))

m = map(lambda x: 2*x, range(10))
select(m, 1)

2

In [19]:
# leave (default: True): keeps all traces of the progressbar upon termination of iteration.
# dynamic_ncols (default: False): if set, constantly alters ncols and nrows to the environment
#    (allowing for window resizes).
k = 0
prefix = 'Test loop: '
for i in tqdm(range(100), desc=prefix, dynamic_ncols=True, leave=True, position=0):
    k += i
    time.sleep(0.01)
k

Test loop: 100%|██████████| 100/100 [00:01<00:00, 97.94it/s]


4950